In [1]:
# LogRISE L2(fields)-L2(edges) for Potts Model
# Optimizer: LBFGS-B
# Required packages: Numpy, Scipy, Pandas, time, matplotlib, Numba

import numpy as np
import pandas as pd
from scipy.optimize import minimize, fmin_l_bfgs_b
from scipy import sparse
from numba import jit
import time


# Read arguments from file
args = pd.read_csv("arguments.csv",header=None)

# Define arguments
lamda_field = args.iloc[0,0]
lamda_edge = args.iloc[0,1]
file_samples_histo = args.iloc[0,2].strip()
file_reconstruction = args.iloc[0,3].strip()

# Redefining L2 regularization value here. Delete when all arguments specified in argument file. 
lamda_field = 10.0
lamda_edge = 10.0

# Create histogram of configurations
samples_histo = pd.read_csv(file_samples_histo,header=None).values

num_conf,num_row = samples_histo.shape # number of unique samples, length of chain+1
num_spins = num_row-1 # length of chain
num_samples = samples_histo[:,0].sum() #frequency of each sample
num_classes = len(np.unique(samples_histo[:,1:])) # number of states
num_params = num_spins*num_classes**2 # number of parameters

# Array to store parameters
reconstruction = np.empty((num_spins,num_spins,num_classes,num_classes))

# Functions to convert between flattened and n-dimensional parameter array
def to1D(x): 
    return x.flatten()

def toND(vec): 
    assert vec.shape[0]==num_params
    return vec.reshape(num_spins,num_classes,num_classes)

# Index matrix function. For each sample chain specifies 1D array with indices of present parameters.
# Size: num_conf x num_spins
@jit(nopython=True)
def define_indexmatrix(current_spin,im_type=np.int32):
    index_matrix = np.zeros((num_conf,num_spins), dtype=im_type)
    for k in range(num_conf):            
        ispin = samples_histo[k,1+current_spin]
        for j in range(num_spins):
            jspin = samples_histo[k,1+j]
            index_matrix[k,j] = int(j*num_classes**2 + (ispin-1)*num_classes + jspin-1)
    return index_matrix

# Bound condition function. Returns list of tuples, each specifying bounds for parameter i. 
# Parameter corresponding to (i,j,A,B) should be set to 0 if i==j, A!=B
def bound_condition(i,current_spin):
    if i>=current_spin*num_classes**2 and i<(current_spin+1)*num_classes**2 and ((i-current_spin*num_classes**2)%num_classes)!=(i-current_spin*num_classes**2)//num_classes:
        return (0,0)
    else:
        return (-np.inf,np.inf) 
    
# Find sample weights according to reweighting scheme in PSICOV/CCMPRED

@jit(nopython=True)
def compute_r(samples_histo,const=0.32*0.38):
    avg = 0
    for i in range(num_conf):
        for j in range(i+1,num_conf):
            avg += (samples_histo[i,1:]==samples_histo[j,1:]).sum()
    num_pairs = num_conf*(num_conf-1)//2
    avg = avg/(num_spins*num_pairs)
    r = const/avg
    return r if r<0.5 else 0.5 #as per PSICOV

@jit(nopython=True)
def compute_weights(samples_histo,r,normalize=True):
    sample_weights = np.zeros(num_conf)
    rL = r*num_spins
    for i in range(num_conf):
        for j in range(num_conf):
            if (samples_histo[i,1:]==samples_histo[j,1:]).sum() >= rL:
                sample_weights[i] += 1 
    sample_weights = 1/sample_weights
    if normalize: sample_weights = sample_weights/sample_weights.sum()
    return sample_weights

#r = compute_r(samples_histo)
r = 0.8 #default r for CCMPRED
#sample_weights = compute_weights(samples_histo,r)
sample_weights = np.ones(num_conf)


In [2]:
total_time = time.time()

for current_spin in range(num_spins): 
    
    print("Optimizing parameters for spin "+str(current_spin))
    spin_time = time.time()        
    
    x0 = np.zeros(num_params) #initial parameter guess
    index_matrix = define_indexmatrix(current_spin)
    
    # Define sparse weight matrix
    wm = sparse.csr_matrix((np.ones(index_matrix.shape[0]*index_matrix.shape[1]), 
                               index_matrix.flatten(), num_spins*np.arange(num_conf+1)),
                              shape=(num_conf,num_params))
    
    # Create regularization vector   
    reg_matrix_l2 = lamda_edge*np.ones(num_params)
    reg_matrix_l2[current_spin*num_classes**2:(current_spin+1)*num_classes**2] = lamda_field
    
    # Define bounds          
    bounds = [bound_condition(i,current_spin) for i in range(num_params)]
    
    # L2 Objective
    def objectivel2(x):

        #Log-Sum-Exp Trick
        reg = np.dot(reg_matrix_l2*x,x)  
        sample_args = -np.sum(np.take(x,index_matrix),axis=1) 
        sample_max = np.max(sample_args)
        sample_args = sample_args - sample_max     
        #fo_unsummed = (samples_histo[:,0]/num_samples)*np.exp(sample_args)
        fo_unsummed = sample_weights*np.exp(sample_args) #Reweighting
        return (sample_max + np.log(np.sum(fo_unsummed)) + reg)
    
    # L2 Jacobian
    def jacobianl2(x):

        sample_args = -np.sum(x[index_matrix],axis=1) 
        sample_max = np.max(sample_args)
        sample_args = sample_args - sample_max

        #fo_unsummed = (samples_histo[:,0]/num_samples)*np.exp(sample_args)
        fo_unsummed = sample_weights*np.exp(sample_args) #Reweighting
        fo_unsummed = fo_unsummed/np.sum(fo_unsummed)

        jac = -wm.transpose().dot(fo_unsummed) + 2.0*reg_matrix_l2*x
        return jac 
    
    # L2 Objective + Jacobian. Can be used if using Scipy fmin_l_bfgs to reduce computation times. 
    def objjacl2(x):

        reg_vec = reg_matrix_l2*x
        reg = np.dot(reg_vec,x) 
        sample_args = -np.sum(x[index_matrix],axis=1) 
        sample_max = np.max(sample_args)
        sample_args = sample_args - sample_max     
        #fo_unsummed = (samples_histo[:,0]/num_samples)*np.exp(sample_args)
        fo_unsummed = sample_weights*np.exp(sample_args) #Reweighting
        f = sample_max + np.log(np.sum(fo_unsummed)) + reg
        fo_unsummed = fo_unsummed/np.sum(fo_unsummed)
        
        jac = -wm.transpose().dot(fo_unsummed) + 2.0*reg_vec
        
        return f, jac
  
    # Optimize
    sol = minimize(objectivel2, x0,jac=jacobianl2,bounds=bounds,method="L-BFGS-B",tol=1e-6)     
    #sol = fmin_l_bfgs_b(objjacl2,x0,fprime=None)
        
    params = toND(sol['x'].copy())
    reconstruction[current_spin,:,:,:] = params.copy()
     
    print("Time Elapsed for spin ",current_spin," : ", time.time()- spin_time)
    print("Success!") if sol['success'] else print("Failed!",sol)   
        
print("Total Time for all spins : ", time.time()-total_time)

Optimizing parameters for spin 0
Time Elapsed for spin  0  :  0.5357089042663574
Success!
Optimizing parameters for spin 1
Time Elapsed for spin  1  :  0.005849123001098633
Success!
Optimizing parameters for spin 2
Time Elapsed for spin  2  :  0.005897998809814453
Success!
Optimizing parameters for spin 3
Time Elapsed for spin  3  :  0.004334211349487305
Success!
Optimizing parameters for spin 4
Time Elapsed for spin  4  :  0.005441904067993164
Success!
Optimizing parameters for spin 5
Time Elapsed for spin  5  :  0.006967067718505859
Success!
Optimizing parameters for spin 6
Time Elapsed for spin  6  :  0.005323886871337891
Success!
Optimizing parameters for spin 7
Time Elapsed for spin  7  :  0.00459599494934082
Success!
Total Time for all spins :  0.5796730518341064


In [10]:
trueJ = np.load("gibbscoupling.npy")
print(np.linalg.norm(reconstruction - trueJ))

1.9171053051170062


In [4]:
trueJ

array([[[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        , -0.27215704],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [-0.86048394,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [5]:
reconstruction

array([[[[0.02112193, 0.        ],
         [0.        , 0.01088078]],

        [[0.01024115, 0.01694264],
         [0.01193697, 0.0120634 ]],

        [[0.00905853, 0.01847609],
         [0.01040352, 0.01334122]],

        [[0.00778071, 0.01639225],
         [0.01248735, 0.01220882]],

        [[0.00891311, 0.01198287],
         [0.01689674, 0.01013277]],

        [[0.01098916, 0.01242419],
         [0.01645541, 0.00550655]],

        [[0.01561538, 0.01505699],
         [0.01382262, 0.00979813]],

        [[0.0113238 , 0.        ],
         [0.        , 0.        ]]],


       [[[0.01015463, 0.02543975],
         [0.        , 0.        ]],

        [[0.02456333, 0.        ],
         [0.        , 0.01427092]],

        [[0.01029242, 0.01638113],
         [0.00905861, 0.01544543]],

        [[0.00911791, 0.01464   ],
         [0.01079974, 0.01396979]],

        [[0.01059354, 0.0107914 ],
         [0.01464834, 0.01131915]],

        [[0.01324418, 0.00945303],
         [0.01598672, 0.008